# Stitch JHU data through various schema changes
* Reshape
* See what columns we need to derive

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
# https://gist.github.com/rogerallen/1583593
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# reverse the dict
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

## Pre 2/14/2020

In [3]:
pre214_cases_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/archived_data/archived_time_series/time_series_2019-ncov-Confirmed.csv"
pre214_deaths_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/archived_data/archived_time_series/time_series_2019-ncov-Deaths.csv"
pre214_recovered_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/archived_data/archived_time_series/time_series_2019-ncov-Recovered.csv" 

In [4]:
cases1 = pd.read_csv(pre214_cases_url)
deaths1 = pd.read_csv(pre214_deaths_url)
recovered1 = pd.read_csv(pre214_recovered_url)

Modified helper function, since columns are datetime, will extract date portion.

In [5]:
def parse_columns(df):
    """
    quick helper function to parse columns into values
    uses for pd.melt
    """
    df.columns = df.columns.str.split(' ').str[0]
    columns = list(df.columns)
    id_vars, dates = [], []

    for c in columns:
        if c.endswith("20"):
            dates.append(c)
        else:
            id_vars.append(c)
    return id_vars, dates

In [6]:
id_vars, dates = parse_columns(cases1)
pre214_df = pd.melt(cases1, id_vars=id_vars, value_vars=dates, value_name="cases", var_name="date",
)

# melt deaths
id_vars, dates = parse_columns(deaths1)
deaths_df = pd.melt(deaths1, id_vars=id_vars, value_vars=dates, value_name="deaths")

# melt recovered
id_vars, dates = parse_columns(recovered1)
recovered_df = pd.melt(
    recovered1, id_vars=id_vars, value_vars=dates, value_name="recovered"
)

# join
pre214_df["deaths"] = deaths_df.deaths
pre214_df["recovered"] = recovered_df.recovered

## Pre 3/23
* will have overlap with pre 2/14...let's see how this resolves itself

In [7]:
pre323_cases_url= "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
pre323_deaths_url= "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv"
pre323_recovered_url= "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv"

In [8]:
cases2 = pd.read_csv(pre323_cases_url)
deaths2 = pd.read_csv(pre323_deaths_url)
recovered2 = pd.read_csv(pre323_recovered_url)

In [9]:
id_vars, dates = parse_columns(cases2)
pre323_df = pd.melt(cases2, id_vars=id_vars, value_vars=dates, value_name="cases", var_name="date",
)

# melt deaths
id_vars, dates = parse_columns(deaths2)
deaths_df2 = pd.melt(deaths2, id_vars=id_vars, value_vars=dates, value_name="deaths")

# melt recovered
id_vars, dates = parse_columns(recovered2)
recovered_df2 = pd.melt(
    recovered2, id_vars=id_vars, value_vars=dates, value_name="recovered"
)

# join
pre323_df["deaths"] = deaths_df2.deaths
pre323_df["recovered"] = recovered_df2.recovered

## Pre 3/23 data is of 2 types
* Pre 3/10 county level...which need to be summed up to get state totals
* 3/10-3/23 state level...lose county level except for SCAG region

### Combine pre214_df and pre310_df and get rid of duplicates

In [10]:
pre310_df = pre323_df[pre323_df.date <= '3/10/20']

combined_df1 = pre214_df.append(pre310_df).sort_values(['Country/Region', 'Province/State', 'date'])


combined_df1.rename(columns = {'Country/Region':'Country_Region', 
                               'Province/State': 'Province_State'}, inplace = True)

In [11]:
combined_df1 = combined_df1.drop_duplicates()

In [12]:
combined_df1['obs'] = combined_df1.groupby(['Province_State', 'Country_Region', 
                                          'Lat', 'Long', 'date']).cumcount() + 1

combined_df1['max_obs'] = combined_df1.groupby(['Province_State', 'Country_Region',
                                             'Lat', 'Long', 'date'])['obs'].transform('max')

In [13]:
# If there are still duplicates, it's because JHU sometimes did multiple updates a day
# This is ok, we'll keep the higher values for cases, deaths, recovered. 
for col in ['cases', 'deaths', 'recovered']:
    combined_df1[col] = combined_df1.groupby(['Province_State', 'Country_Region', 
                                            'Lat', 'Long', 'date'])[col].transform('max').fillna(0).astype(int)

combined_df1 = combined_df1.drop_duplicates(subset = ['Province_State', 'Country_Region',
                                                   'Lat', 'Long', 'date',
                                                   'cases', 'deaths', 'recovered'], keep = 'last')

combined_df1 = combined_df1.drop(columns = ['obs', 'max_obs'])

In [14]:
combined_df1[(combined_df1.Country_Region == 'Australia') & 
             (combined_df1.date >= '3/1/20')].sort_values('date').head(10)

,Province_State,Country_Region,Lat,Long,date,cases,deaths,recovered
19941,Australian Capital Territory,Australia,-35.4735,149.0124,3/1/20,0,0,0
19546,Victoria,Australia,-37.8136,144.9631,3/1/20,7,0,4
19600,Tasmania,Australia,-41.4545,145.9707,3/1/20,0,0,0
19558,South Australia,Australia,-34.9285,138.6007,3/1/20,3,0,2
19588,Western Australia,Australia,-31.9505,115.8605,3/1/20,2,1,0
19610,Northern Territory,Australia,-12.4634,130.8456,3/1/20,0,0,0
19547,Queensland,Australia,-28.0167,153.4000,3/1/20,9,0,1
19545,New South Wales,Australia,-33.8688,151.2093,3/1/20,6,0,4
19561,From Diamond Princess,Australia,35.4437,139.6380,3/1/20,0,0,0
24119,Northern Territory,Australia,-12.4634,130.8456,3/10/20,1,0,0


In [15]:
# For the US, since Province/State contains county and state info, we need to derive our own state totals
us1 = combined_df1[combined_df1.Country_Region == "US"]

world1 = combined_df1[combined_df1.Country_Region != "US"]

# Drop observations that don't include a comma, these are state observations, but JHU filled with zeros
us1 = us1[us1.Province_State.str.contains(',')]

In [16]:
# Grab state abbrev
us1['state_abbrev'] = us1.Province_State.str.split(', ').str[1].str.replace('.', '')

In [17]:
# Calculate US State totals
def us_state_totals(df):
    state_totals = df.groupby(['Country_Region', 'state_abbrev']).agg(
        {'cases':'sum', 'recovered':'sum', 'deaths':'sum'})
    
    state_totals.rename(columns = {'cases': 'state_cases',
                                  'recovered':'state_recovered', 
                                  'deaths': 'state_deaths'}, inplace = True)
    
    df = pd.merge(df, state_totals, on = ['Country_Region', 'state_abbrev'])
    
    return df

# Calculate non-US Province_State totals
def province_totals(df):
    province_totals = df.groupby(['Country_Region', 'Province_State']).agg(
        {'cases':'sum', 'recovered':'sum', 'deaths':'sum'})
    
    province_totals.rename(columns = {'cases': 'state_cases',
                                  'recovered':'state_recovered', 
                                  'deaths': 'state_deaths'}, inplace = True)
    
    df = pd.merge(df, province_totals, on = ['Country_Region', 'Province_State']) 
    
    return df

# Calculate country totals
def country_totals(df):
    country_totals = df.groupby(['Country_Region']).agg(
        {'cases':'sum', 'recovered':'sum', 'deaths':'sum'})
    
    country_totals.rename(columns = {'cases': 'country_cases',
                                  'recovered':'country_recovered', 
                                  'deaths': 'country_deaths'}, inplace = True)
    
    df = pd.merge(df, country_totals, on = ['Country_Region']) 
    
    return df

In [18]:
us1 = us_state_totals(us1)

In [21]:
us1[(us1.state_abbrev=='CA') & (us1.date >= '3/10/20')]

,Province_State,Country_Region,Lat,Long,date,cases,deaths,recovered,state_abbrev,state_cases,state_recovered,state_deaths
901,"Alameda County, CA",US,37.6017,-121.7195,3/10/20,0,0,0,CA,297,20,0
942,"Calaveras, CA",US,38.1960,-120.6805,3/10/20,0,0,0,CA,297,20,0
983,"Contra Costa County, CA",US,37.8534,-121.9018,3/10/20,0,0,0,CA,297,20,0
1024,"Fresno County, CA",US,36.9859,-119.2321,3/10/20,0,0,0,CA,297,20,0
1065,"Humboldt County, CA",US,40.7450,-123.8695,3/10/20,0,0,0,CA,297,20,0
1107,"Los Angeles, CA",US,34.0522,-118.2437,3/10/20,0,0,0,CA,297,20,0
1148,"Madera County, CA",US,37.2519,-119.6963,3/10/20,0,0,0,CA,297,20,0
1189,"Marin, CA",US,38.0834,-122.7633,3/10/20,0,0,0,CA,297,20,0
1230,"Napa, CA",US,38.5025,-122.2654,3/10/20,0,0,0,CA,297,20,0
1271,"Orange County, CA",US,33.7879,-117.8531,3/10/20,0,0,0,CA,297,20,0


In [ ]:
us1 = country_totals(us1)

world1 = us_state_totals(world1)
world1 = country_totals(world1)

## Correctly append pre214_df and pre323_df
* There are some zeroes JHU filled in, throw those out
* Derive state-level counts

In [ ]:
combined_df = pre214_df.append(pre323_df).sort_values(['Country/Region', 'Province/State', 'date'])

In [ ]:
# Drop duplicates
combined_df = combined_df.drop_duplicates()

In [ ]:
combined_df['obs'] = combined_df.groupby(['Province/State', 'Country/Region', 
                                          'Lat', 'Long', 'date']).cumcount() + 1

combined_df['max_obs'] = combined_df.groupby(['Province/State', 'Country/Region',
                                             'Lat', 'Long', 'date'])['obs'].transform('max')

In [ ]:
# If there are still duplicates, it's because JHU sometimes did multiple updates a day
# This is ok, we'll keep the higher values for cases, deaths, recovered. 
for col in ['cases', 'deaths', 'recovered']:
    combined_df[col] = combined_df.groupby(['Province/State', 'Country/Region', 
                                            'Lat', 'Long', 'date'])[col].transform('max').fillna(0).astype(int)

combined_df = combined_df.drop_duplicates(subset = ['Province/State', 'Country/Region',
                                                   'Lat', 'Long', 'date',
                                                   'cases', 'deaths', 'recovered'], keep = 'last')

combined_df = combined_df.drop(columns = ['obs', 'max_obs'])

In [ ]:
combined_df.head()

## Post 3/23 feature layer

In [ ]:
feature_layer_url = "https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/ncov_cases_US/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=OBJECTID%2C+Province_State%2C+Country_Region%2C+Last_Update%2C+Lat%2C+Long_%2C+Confirmed%2C+Recovered%2C+Deaths%2C+Active%2C+Admin2%2C+FIPS%2C+Combined_Key%2C+Incident_Rate%2C+People_Tested&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="

cases3 = gpd.read_file(feature_layer_url)

#cases3.to_file(driver = 'GeoJSON', filename = '../data/jhu_feature_layer_3_25_2020.geojson')

In [ ]:
# Last_Update column is displaying weird...
cases3.head()

In [ ]:
cases3['date'] = '3/25/2020'
cases3['date'] = pd.to_datetime(cases3.date)

In [ ]:
# Admin2 = County
# Province_State = US State
# Combined_Key = County, State, Country
cases3['county_state'] = cases3.Combined_Key.str.slice(0, -4)

In [ ]:
keep_identifiers = ['Combined_Key', 'FIPS']
geog = cases3[keep_identifiers]

In [ ]:
geog['county'] = geog.Combined_Key.str.split(',', expand = True)[0].str.strip()
geog['state'] = geog.Combined_Key.str.split(',', expand = True)[1].str.strip()

In [ ]:
geog['state_abbrev'] = geog.state.map(us_state_abbrev)

geog['orig_county'] = geog.county + ", " + geog.state_abbrev

geog = geog.drop(columns = ['county', 'state'])

In [ ]:
cases3 = pd.merge(cases3, geog, on = ['Combined_Key', 'FIPS'])

## Make combined_df the same as cases3

In [ ]:
cases3.rename(columns = {"Long_":"Lon", 
                        "Confirmed":"cases", 
                        "Recovered":"recovered", 
                        "Deaths":"deaths", 
                        "Admin2": "County"} , inplace = True)

combined_df.rename(columns = {"Province/State":"Province_State", 
                             "Country/Region":"Country_Region", 
                             "Long":"Lon"}, inplace = True)

In [ ]:
us = combined_df[combined_df.Country_Region == "US"]
world = combined_df[combined_df.Country_Region!="US"]

In [ ]:
# Create new column that stores county information (County, StateAbbrev) in orig_county to match with cases3
us['orig_county'] = us.Province_State

In [ ]:
# Change Province_State to display the full state name
us['Province_State'] = us.Province_State.str.split(', ').str[1]

us['Province_State'] = us.Province_State.map(abbrev_us_state)

In [ ]:
us.head()

In [ ]:
# Add Combined_Key as column
us['county'] = us.orig_county.str.split(', ').str[0]

us['Combined_Key'] = us.county + ", " + us.Province_State + ", " + us.Country_Region

In [ ]:
# Add FIPS, state_abbrev, Combined_Key columns
us['state_abbrev'] = us.Province_State.map(us_state_abbrev)

county_fips_crosswalk = cases3[['orig_county', 'FIPS']].drop_duplicates()

us2 = pd.merge(us, county_fips_crosswalk, on = 'orig_county')

In [ ]:
combined_df2 = us2.append(world)

## Append combined_df2 and cases3

In [ ]:
df = combined_df2.append(cases3)

keep_cols = ['Province_State', 'Country_Region', 'date', 
             'Lat', 'Lon', 'cases', 'recovered', 'deaths',
            'FIPS', 'Combined_Key', 'Incident_Rate', 'People_Tested', 'geometry', 
            'state_abbrev', 'orig_county']

df = df[keep_cols]

In [ ]:
from geoalchemy2 import Geometry, WKTElement
from shapely.geometry import Point

# Fill in geometry - may not be necessary because we ESRI uses Lat, Lon columns
srid = 4326
df = df.dropna(subset=['Lat', 'Lon'])
df["geometry"] = df.apply(
    lambda x: WKTElement(Point(x.Lon, x.Lat).wkt, srid=srid), axis=1
)

In [ ]:
# Change date column to be datetime....Ian has code for this

## Derive new columns: state and country totals

In [ ]:
state_total = df.groupby(['Country_Region', 'Province_State', 'date']).agg({'cases':'sum', 'recovered': 'sum', 
                                                                            'deaths': 'sum'}).reset_index()

state_total.rename(columns = {'cases': 'state_cases', 'recovered': 'state_recovered', 
                              'deaths': 'state_deaths'}, inplace = True)

country_total = df.groupby(['Country_Region', 'date']).agg({'cases':'sum', 'recovered': 'sum', 
                                                            'deaths': 'sum'}).reset_index()

country_total.rename(columns = {'cases': 'country_cases', 'recovered': 'country_recovered', 
                                'deaths': 'country_deaths'}, inplace = True)

In [ ]:
df1 = pd.merge(df, state_total, on = ['Country_Region', 'Province_State', 'date'])
df2 = pd.merge(df1, country_total, on = ['Country_Region', 'date'])

In [ ]:
# There is missing info at the county-level between 3/10-3/23. We can plug in SCAG counties.

In [ ]:
# Where is state information from 3/10-3/23?


In [ ]:
ca = df2[df2.state_abbrev == "CA"]
ca['date'] = pd.to_datetime(ca.date)

In [ ]:
ca[['Province_State', 'date', 'orig_county',
    'cases', 'state_cases', 'country_cases']].to_excel('../data/test_ca.xlsx')